# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_modes.ipynb
- template script for analyzing modes spectra for sumatra

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy import read
from obspy.core import UTCDateTime
from obspy.geodetics import gps2dist_azimuth
from obspy.geodetics import kilometers2degrees

from pysep import Pysep

from lib_seis import station_map_and_table
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft
from lib_seis import w2fstack

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

origin time = 2004-12-26 00:58:53 \
latitude    = 3.09 N \
longitude   = 94.26 E \
depth       = 30 km \
magnitude   = 9.1

In [ ]:
# setting event and data parameters

origin_time        = UTCDateTime('2004,12,26,00,58,53')
event_latitude     = 3.09
event_longitude    = 94.26
event_depth_km     = 30
event_magnitude    = 9.1
resample_freq      = 1
data_dir           = 'datawf/sumatra_LHZ_acc/SAC'

In [ ]:
# read downloaded LHZ acceleration data for selected station locations

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# listing selected waveforms

picked_waveforms = [['G' ,'CAN' ,''  ,'LHZ'],
                    ['II','NNA' ,'00','LHZ'],
                    ['IU','BBSR','00','LHZ'],
                    ['II','SUR' ,'00','LHZ']]

##############################################
####### PASTE COMPLETE LIST ABOVE ############
##############################################

In [ ]:
# station map and distances table for the selected station locations
station_map_and_table(st, st_subset_list=picked_waveforms, event_lat=event_latitude, event_lon=event_longitude)

In [ ]:
# trimming of waveforms which can be used after trimming

waveforms_to_reject, waveforms_to_trim = sumatra_waveform_screening(channel='LHZ')

for i, trim_details in enumerate(waveforms_to_trim):
    ID = trim_details[0]
    
    try:
        tr = st.select(id=ID)
        start_time = tr[0].stats.starttime + trim_details[1] 
        end_time   = tr[0].stats.starttime + trim_details[2]
        tr.trim(start_time,end_time)
    except:
        pass

In [ ]:
# computing and creating a list of the amplitude spectra of the seismograms for the selected station locations

amps  = []
freqs = []

Nyquist_frequency = resample_freq/2

for i, waveform_id in enumerate(picked_waveforms):
    
    ID = f'{waveform_id[0]}.{waveform_id[1]}.{waveform_id[2]}.{waveform_id[3]}'
    tr = st.select(id=ID)

    tr.detrend('constant')
    tr.detrend('linear')
    tr.taper(max_percentage=0.2, type="cosine")
    
    fft_amp, fft_phase, fft_freq = wf_fft(tr[0].data, Nyquist_frequency)
    
    freqs.append(fft_freq)
    amps.append(fft_amp)

In [ ]:
# example of stacking spectra

f1 = 0.2 * 1E-3
f2 = 10  * 1E-3
n  = 100000

amp_stack, freq, amplitudes = w2fstack(freqs,amps,f1,f2,n)                                                     

plt.figure()
plt.plot(freq*1E3, amp_stack, 'b') 
plt.title('Stacked of amplitude spectra of selected waveforms')
plt.xlabel('Frequency [mHz]')
plt.ylabel('Amplitude ')
plt.show()